---
title: Week 7 Real world Application 01, Simulation of Travertine formation
subtitle: Orchestra Scenario, Lorah & Herman
author:
  - name: Timo Heimovaara
    affiliations: Delft University of Technology, department of Geoscience & Engineering
    orcid: 
    email: t.j.heimovaara@tudelft.nl
license: CC-BY-NC-ND-4.0 (https://creativecommons.org/licenses/by-nc-nd/4.0/).
date: 2026-01-30
kernelspec:
    name: python3
    display_name: 'Python 3.13'
---

## Travertine deposition along a stream
In this assignment you will use Orchestra to model the precipitation of Travertine along a stream. The data we have comes from a paper by [Lorah and Herman, 1988](doi:10.1029/WR024i009p01541) which you can download by clicking on the reference.

In [1]:
# Import libraries required for running all simulations
import os
import sys
from IPython.utils import capture
from IPython.display import display, Markdown
from pathlib import Path
from contextlib import chdir

import numpy as np
import matplotlib.pyplot as plt
import PyORCHESTRA # here, the ORCHESTRA submodule is imported
import pandas as pd
import seaborn as sns

%matplotlib widget
sns.set()

# Prepare a file to capture PyOrchestra output
capture_file = open("pyorchestra_output.log", "w")


# pyOrchestra is implemented in C++
# Save original stdout file descriptor
original_stdout_fd = sys.stdout.fileno()

# Duplicate original stdout so we can restore it later
saved_stdout_fd = os.dup(original_stdout_fd)



# We need to import some Orchestra files. We need to know the path layout on 
# the local machine:
def find_book_root(start: Path | None = None) -> Path:
    """
    Walk upward from `start` (or CWD) until a directory containing Jupyter Book
    marker files is found. Returns the path to the book root.
    Raises FileNotFoundError if no root is found.
    """
    config_any = {"_config.yml", "_config.yaml"}      # some projectrs use .yaml
    myst_any = {"myst.yml", "myst.yaml"}            # jupyter-book uses _toc.yml

    cur = Path(Path.cwd()).resolve()

    for parent in [cur, *cur.parents]:
        children = {f.name for f in parent.iterdir()} if parent.exists() else set()
        has_any_config = bool(config_any & children)
        has_any_myst = bool(myst_any & children)
        if has_any_config and has_any_myst:
            return parent

    raise FileNotFoundError(
        f"Could not find Jupyter Book root (no _config.y* and myst.y* found above {cur})"
    )


def path_from_book_root(*parts: str | Path) -> Path:
    root = find_book_root()
    p = (root.joinpath(*parts)).resolve()
    if not p.exists():
        raise FileNotFoundError(f"Path not found: {p}")
    return p

# In order for orchestra to run we need to change directory to the directory with the input file:


# Example usage:
# input_file = path_from_book_root("content", "week 02", "Orchestra_simulation", "chemistry1.inp")
# print("Input file:", input_file)

orchestra_path = path_from_book_root("content", "week 3.7", "Assignment_Lorah_and_Herman")
# print(orchestra_path)

## Download this script and the required ORCHESTRA files
TODO Create an yml for a local python environment
TODO Create zip file with the correct files. Explain how to install Orchestra using pip in a local python environment.
TODO Update all files so that they are well documented and describe the system

pyorchestra will be initialize with the correct chemistry file...

explain that InVars are the species used to define the totals. 
explain that OutVars is the list of Orchestra variables that are exported to python for post-processing or necessary for simulation.

Once prinicple problem is known we can create and initialize a pyorchestra object.

## Unravelling the geochemistry of Travertine deposition along a stream
In the paper by Lorah and Herman (1988) you will see that several sampling trips were made to collect samples along a stream moving away from the hotsprings where the stream originates, past a waterfall and then further down stream. In Table 1, analysis results are presented for to sampling campaigns, one on 14 October 1984 and another on 6  April 1985. We will use these data for our analysis.

The aim of the analysis is to increase our understanding of the formation of travertine deposits along the stream and especially near the water fall.

The analysis you will do follows the approach given in the paper and consists in principle of two main steps:
1. We use that analysis results from the table to assess the the most likely in-situ conditions at the moment of sampling. The analysis results, give totals of the elements expressed as shown in the header of the table. The true speciation will vary, elements will be present in other forms as well, solid minerals may control the composition of the water, but are not sampled with the water. The system may, or may not be in equilibrium. We analyse this problem with Orchestra using an Chemistry input file where we give the total of all measured compounds as master species, and we do not allow any minerals to precipitate, but we do report the SI-values for those minerals.
2. After doing the first analysis, we have quite alot of additional information compared to the chemical analysis results from the lab, we know partial $\text{CO}_2[g]$ pressure and the SI-values for possible minerals for the water samples along the river. It is safe to assume that water down stream from the spring originated from the spring and that its composition should be realated to the water at the spring. With this assumption we can use Orchestra to simulate changes related to the measurements obtained above. For example: if we have water from S-1 and move this to the conditions at D-3 ($\text{CO}_2[g]$ pressure and Calcite-SI), how much calcite would have precipitated in between?

In Orchestra you define the type of calculation you want to do with a *chemistry.inp* file. For this assignment we need two types:
1. To calculate the equilibrium calculations for our system without allowing minerals to precipitate;
2. As similar input file, but where we allow the minerals to precipitate. As we want to compare samples along the line, we also need to be able to control the SI-value which is used to achieve the equilibrium. Normally, the SI-value is assumed to be zero for supersaturated samples, in this assignment we need to allow the model to equilibrate assuming higher, or even lower SI-values. In addition, the $\text{CO}_2[g]$ pressure needs to be fixed as well, which we can achieve by fixing the CO2[g].logact.



```{exercise} **Assumptions**
As stated above, we assume that the composition of the water downstream of the spring is closely related to the composition of the water at the stream. Please carefully evaluate this assumption and try to find reasons why this assumption might not be true? What can you do to mitigate the reasons you found?
```

## Step 1: Assessing the water samples
For this first step I have already prepared an Orchestra input file using the *orchestra2026.jar* Graphical User Interface (GUI). Please have a look at this file with the GUI. 
The approach was quite simple, I have added all the analysed compounds that are found in the table as primary entities in the dissolved phase (*diss*). We define the pH as a fixed logact and we assume that water is the solute phase with a fixed logactivity of 0. We do not select any minerals on the *Phases & Reactions* tab, because we do not allow for any precipitation. We assume that the water samples are stored in bottles with out any gas-phase (head space), we set the gasvolume to be zero on the *Variables* tab. This chemical system is defined in *chemistry1.inp*.

In order to illustrate the steps, we will analyse two water samples: S1 and F1. Please note that the concentrations are given in $\text{mg/l}$ and the temperature in $^\text{o}\text{C}$, we need to traslate these numbers to $\text{mol/l}$ and $\text{K}$ in order to work with Orchestra.

```{hint} Creating the *chemistry.inp* file
Using the Orchestra GUI shown in [](#master_species)  we are able to define a system of the primary entities. You do this by selecting the **Chemistry** tab on the right in the ORCHESTRA-COMPOSER window. The **Chemistry** tab shows a number of tabs which fill the *chemistry1.inp* file. We start by selecting tab **Primary entities/ Master Species**. Remember that we select the Master Species in such a way that we can create all other species for our problem. In addition we need to use these species to define the total moles of each master species present in the system (including the amounts of these master species present in the secondary species).
```


## Initialise the problem 
In order solve this problem with pyOrchestra we first initialize our problem using the *chemistry1.inp* file. We also specify the inputs and outputs which are passed to and from Orchestra. The inputs are from the master species (or primary entities). The outputs can be anything we are interested in.

Once pyOrchestra is initialized, running a simulation consists of a series of steps where the values of the required set of input variables are passed via *InVARS* to ORCHESTRA, after which a set of corresponding output variables are passed back in *OutVars*.

ORCHESTRA is initialized in pyOrchestra using the inputfile *chemistry1.inp*, created above with the ORCHESTRA-GUI. After initialization in Python, we know which variables will be passed through *OutVars* and can be used in *InVars*.

The following code implements these steps.


In [2]:
#--- Initialize problem --- 
# for initialization we need to temporarily move to the directory containing the 'chemistry1.inp' file.
# Later we use pO1, InVars1 and OutVars1
with chdir(orchestra_path):

    # Input file is generated with Orchestra GUI
    InputFile = 'chemistry1.inp'
    NoCells = 1 #only 1 cell to have a 0-D system with 1liter of water
    
    # We define the input variables that will be changed in the script
    # We use the same sequence as used in the paper
    InVars1 = np.array(['T','pH', 'HCO3-.tot', 'Ca+2.tot',  'Mg+2.tot',
                       'Na+.tot', 'K+.tot', 'F-.tot', 'Cl-.tot', 'SO4-2.tot',
                       'watervolume', 'gasvolume'
                       ])
    
    # We select the output from Orchestra we need to use
    OutVars1 = np.array(['T','pH', 
                        'HCO3-.tot', 'HCO3-.con', 'HCO3-.logact', 'HCO3-.diss',
                        'H2CO3.tot', 'H2CO3.con', 'H2CO3.logact', 'H2CO3.diss',
                        'CO3-2.tot', 'CO3-2.con', 'CO3-2.logact', 'CO3-2.diss',
                        'CO2[g].tot', 'CO2[g].con', 'CO2[g].logact', 'CO2[g].diss',
                        'SO4-2.tot', 'SO4-2.con', 'SO4-2.logact','SO4-2.diss',
                        'HSO4-.tot', 'HSO4-.con', 'HSO4-.logact',
                        'Ca+2.tot', 'Ca+2.con', 'Ca+2.logact', 'Ca+2.diss', 
                        'CaF+.con', 'CaOH+.con', 'CaSO4.con',
                        'Calcite[s].si','Calcite[s].tot','Calcite[s].logact',
                        'Fluorite[s].si','Fluorite[s].tot','Fluorite[s].logact',
                        'Gypsum[s].si', 'Gypsum[s].tot','Gypsum[s].logact',
                        'I', 'chargebalance', 'watervolume' ])

    # Associate a variable with the pyOrchestra.ORCHESTRA() class
    pO1 = PyORCHESTRA.ORCHESTRA()

    # Initialize the class with the parameters defined above
    pO1.initialise(InputFile, NoCells, InVars1, OutVars1)


Reading and expanding calculator new stylechemistry1.inp
Scanning file: chemistry1.inp
Scanning file: objects2025.txt
Scanning file: chemistry1.inp
Scanning file: objects2025.txt
Including file: chemistry1.inp
Scanning file: objects2025.txt
0.049 sec.
	Reading variables .... 0.027 s
testing:
4:T
12:pH
14:HCO3-.tot
7:Ca+2.tot
18:Mg+2.tot
20:Na+.tot
16:K+.tot
11:F-.tot
9:Cl-.tot
22:SO4-2.tot
23:watervolume
24:gasvolume
4:T
12:pH
14:HCO3-.tot
25:HCO3-.con
13:HCO3-.logact
26:HCO3-.diss
27:H2CO3.tot
28:H2CO3.con
29:H2CO3.logact
30:H2CO3.diss
31:CO3-2.tot
32:CO3-2.con
33:CO3-2.logact
34:CO3-2.diss
35:CO2[g].tot
36:CO2[g].con
37:CO2[g].logact
38:CO2[g].diss
22:SO4-2.tot
39:SO4-2.con
21:SO4-2.logact
40:SO4-2.diss
41:HSO4-.tot
42:HSO4-.con
43:HSO4-.logact
7:Ca+2.tot
44:Ca+2.con
6:Ca+2.logact
45:Ca+2.diss
46:CaF+.con
47:CaOH+.con
48:CaSO4.con
49:Calcite[s].si
50:Calcite[s].tot
51:Calcite[s].logact
52:Fluorite[s].si
53:Fluorite[s].tot
54:Fluorite[s].logact
55:Gypsum[s].si
56:Gypsum[s].tot
57:Gyps

Please note that the output of this code is what ORCHESTRA echos back. ORCHESTRA uses a set of variables in order to store the input variables and the results of the calculations, in this case 33. The top part of the output shows the output requested by us through *OutVars* together with the values used during initialization.

### Run the Problem
We analyse two water samples: S1 and F6. Please note that the concentrations are given in mg/l and the temperature in $^o$C, we need to traslate these numbers to mol/l and K in order to work with Orchestra.

### Step 1: import data from paper and create input dataframe for simulations
There are different approaches to import the data. You can type it in to the script directly, you can also copy from the pdf and then edit in the script, or you first copy to a spreadsheet and then work from there. It is a matter of personal taste. The spreadsheet approach, is a good way to move forward as you create a digital backup of the data at the same time.

### Step 2: Translate the data so that we have the temperature in K and the concentrations in mol/l

### Step 3: Copy the values from the imported data to IN1[0] and run Orchestra

## Step 4: Evaluate the output.

In [3]:
# copy data from Lorah and Herman (1988)
# We store the data in a data frame
# Because the data are given in mg/L we need the molar masses to be able to 
# translate to mol/liter

# for conveniece we set mmass for non chemical values to 0.001 because it is then very easy
# to calculate the concentrations in mol per liter...
# Please note: we use Orchestra notation

headers = ['Sample','T_celsius', 'Conductivity', 'pH', 'HCO3-.tot', 'Ca+2.tot',  
           'Mg+2.tot', 'Na+.tot', 'K+.tot', 'F-.tot', 'Cl-.tot', 'SO4-2.tot']
data = [
       ['mmass', 1e-3, 1e-3, 1e-3, 61.02, 40.08, 24.31, 23, 39.1, 19, 34.45, 96.07],
       ['S-1', 24.3, 700, 7.32, 308., 165., 25.4, 5.1, 14.2,  1.0, 3.3, 283.],
       ['F-6', 14.4, 442, 8.25, 210, 122, 21.3, 3.3, 11.4, 0.7, 4, 246]
    ]
df_data_orig = pd.DataFrame(data=data,columns=headers)
df_data_orig.set_index('Sample',inplace=True)

# calculate molar concentrations
df_data = pd.DataFrame()
df_data = df_data_orig.loc[['S-1','F-6']]/df_data_orig.loc['mmass'] * 0.001
# calculate temperature in K
df_data['T'] = df_data['T_celsius'] + 273.15

print(df_data)

        T_celsius  Conductivity    pH  HCO3-.tot  Ca+2.tot  Mg+2.tot  \
Sample                                                                 
S-1          24.3         700.0  7.32   0.005048  0.004117  0.001045   
F-6          14.4         442.0  8.25   0.003441  0.003044  0.000876   

         Na+.tot    K+.tot    F-.tot   Cl-.tot  SO4-2.tot       T  
Sample                                                             
S-1     0.000222  0.000363  0.000053  0.000096   0.002946  297.45  
F-6     0.000143  0.000292  0.000037  0.000116   0.002561  287.55  


In [4]:
# %%
# Run the model using chemistry1.inp for the above samples
IN1 = np.array([np.ones_like(InVars1)]).astype(float)
# prepare output matrix
all_Res = np.zeros([len(df_data),len(OutVars1)])

# set default watervolume and gasvolume
IN1[0][np.where(InVars1 == 'watervolume')] = 1.0 # per liter
IN1[0][np.where(InVars1 == 'gasvolume')] = 0.0 # no gas 

# loop over available samplesCa_Range 
# we use a counter ii in order to store the results
for ii in range(len(df_data)):
    # set InVars
    # please note InVars was initialized assuming the same frequency as
    # Lorah and Herman. 

    # please note, we fill until -2, watervolume and gasvolume have already
    # been defined above. Please note we use "Orchestra" notation
    IN1[0][:-2] = df_data.iloc[ii][['T', 'pH', 'HCO3-.tot', 'Ca+2.tot', 'Mg+2.tot', 
        'Na+.tot', 'K+.tot', 'F-.tot', 'Cl-.tot', 'SO4-2.tot']].values
    
    # run ORCHESTRA
    OUT = pO1.set_and_calculate(IN1)
    all_Res[ii] = OUT[0]

    # Create a dataframe from all_Res
Res_Simulation = pd.DataFrame(all_Res,columns=OutVars1, index=df_data.index)

print(Res_Simulation[['T','pH', 'HCO3-.con', 
                      'Ca+2.con', 'CO2[g].con','CO2[g].logact', 
                      'Calcite[s].si', 'Gypsum[s].si', 'Fluorite[s].si']])

                 T    pH  HCO3-.con  Ca+2.con  CO2[g].con  CO2[g].logact  \
Sample                                                                     
S-1     297.450012  7.32   0.004458  0.003413    0.012252      -1.911809   
F-6     287.549988  8.25   0.003229  0.002523    0.000909      -3.041655   

        Calcite[s].si  Gypsum[s].si  Fluorite[s].si  
Sample                                               
S-1          0.369630     -0.940827       -0.786400  
F-6          0.903334     -1.072058       -1.062087  


## Interpretation of the results
The results clearly show that the water samples are not in equilibrium with the atmosphere and calcite. The partial pressure of $\text{CO}_2[g]$ is about 400 ppm which would be 0.0004 at which is lower that calculated above. In addition the SI for calcite is above zero.

What is also striking to see is that the SI for calcite increases along the stream, the CO2[g].logact decreases along the stream and we know that Calcite is precipitating from the stream. How much calcite will have precipitated moving from S-1 to F-6?

## Step 2: Calculate amount of calcite precipitated along stream from S-1 to F-6
In order to run this scenario, we need a different type of calculation than used for step 1. The main difference is that we need to allow Orchestra to precipitate calcite. We need to activate the precipitation reactions in the GUI, which allows Orchestra to add the reactions to the equation set. In addition we need allow our Python script to fix the CO2[g].logact value, instead of defining the HCO3-.tot amount.

```{exercise} Explain
Why do we need to fix the CO2[g].logact value instead of defining the HCO3-.tot amount?
```
In addition to allowing Calcite to precipitate, we also want to be able to adjust the SI value to control the equilibrium condition of Calcite. We can achieve this by adding a constant to the precipitation reaction of Calcite in the *chemistry.inp* file. In this case we added *deltaSIcalcite* to the file, with a default value of 0. If we set to another value, the model solves an equilibrium for this other value. These changes have been implemented in the *chemistry2.inp* file. We will allow Orchestra to change the pH in order to achieve charge balance.

We can now initialise a second Orchestra calculator for this input file.


In [5]:
#--- Initialize problem --- 
# for initialization we need to temporarily move to the directory containing the 'chemistry2.inp' file.
# Later we use pO2, InVars2 and OutVars2
with chdir(orchestra_path):

    # Input file is generated with Orchestra GUI
    InputFile = 'chemistry2.inp'
    NoCells = 1 #only 1 cell to have a 0-D system with 1liter of water
    
    # We define the input variables that will be changed in the script
    # We use the same sequence as used in the paper
    InVars2 = np.array(['T','CO2[g].logact', 'Ca+2.tot',  'Mg+2.tot',
                       'Na+.tot', 'K+.tot', 'F-.tot', 'Cl-.tot', 'SO4-2.tot',
                       'deltaSIcalcite',
                       'watervolume', 'gasvolume'
                       ])
    
    # We select the output from Orchestra we need to use
    OutVars2 = np.array(['T','pH', 
                        'HCO3-.tot', 'HCO3-.con', 'HCO3-.logact', 'HCO3-.diss',
                        'H2CO3.tot', 'H2CO3.con', 'H2CO3.logact', 'H2CO3.diss',
                        'CO3-2.tot', 'CO3-2.con', 'CO3-2.logact', 'CO3-2.diss',
                        'CO2[g].tot', 'CO2[g].con', 'CO2[g].logact', 'CO2[g].diss',
                        'SO4-2.tot', 'SO4-2.con', 'SO4-2.logact','SO4-2.diss',
                        'HSO4-.tot', 'HSO4-.con', 'HSO4-.logact',
                        'Ca+2.tot', 'Ca+2.con', 'Ca+2.logact', 'Ca+2.diss', 
                        'CaF+.con', 'CaOH+.con', 'CaSO4.con',
                        'Calcite[s].si','Calcite[s].tot','Calcite[s].logact',
                        'Fluorite[s].si','Fluorite[s].tot','Fluorite[s].logact',
                        'Gypsum[s].si', 'Gypsum[s].tot','Gypsum[s].logact',
                        'I', 'chargebalance', 'watervolume' ])

    # Associate a variable with the pyOrchestra.ORCHESTRA() class
    pO2 = PyORCHESTRA.ORCHESTRA()

    # Initialize the class with the parameters defined above
    pO2.initialise(InputFile, NoCells, InVars2, OutVars2)

Try a first calculation with iia switched off....
Parsing expressions of chemistry1.inp..... 
Optimizing expressions of chemistry1.inp..... 0.18 sec.
3484 variables, 9913 expressions, 8 equations.
First calculation was successful!
Repeat calculation with iia switched on..
Switching on: logI: -2
This was successful!!
Reading and expanding calculator new stylechemistry2.inp
Scanning file: chemistry2.inp
Scanning file: objects2025.txt
Scanning file: chemistry2.inp
Scanning file: objects2025.txt
Including file: chemistry2.inp
Scanning file: objects2025.txt
0.085 sec.
	Reading variables .... 0.051 s
testing:
4:T
7:CO2[g].logact
9:Ca+2.tot
18:Mg+2.tot
20:Na+.tot
16:K+.tot
13:F-.tot
11:Cl-.tot
22:SO4-2.tot
24:deltaSIcalcite
25:watervolume
26:gasvolume
4:T
14:pH
27:HCO3-.tot
28:HCO3-.con
29:HCO3-.logact
30:HCO3-.diss
31:H2CO3.tot
32:H2CO3.con
33:H2CO3.logact
34:H2CO3.diss
35:CO3-2.tot
36:CO3-2.con
37:CO3-2.logact
38:CO3-2.diss
39:CO2[g].tot
40:CO2[g].con
7:CO2[g].logact
41:CO2[g].diss
22:SO4-2

## Strategy to solve the problem
After initializing the Orchestra Calculator, we can solve the problem using our standard approach:
### 1. Define the initial conditions
The initial condition is determined by the measured (total) concentrations in the water sample at S-1.
### 2. Define the boundary conditions
This is the crucial step for the scenario. The sample at F-6 clearly is in a different condition that S-1. The SI value for calcite is different, the CO2-pressure (CO2[g].con, or CO2[g].logact) is different and finally so is the temperature.

We can set these conditions using the InVars2 array, where we use the masses from S-1 and  T, CO2[g].logact and SI Calcite from F-6 all estimated in the previous calculation. The SI-Calcite calculated for F-6 is used for the deltaSIcalcite variable.

We only carry out one simulation.

In [6]:
# Step 1, initial condition.
# InVars need to contain floats
IN2 = np.array([np.ones_like(InVars2)]).astype(float)
# prepare output matrix
all_Res = np.zeros([1,len(OutVars2)])

# set default watervolume and gasvolume
IN2[0][np.where(InVars2 == 'watervolume')] = 1.0 # per liter
IN2[0][np.where(InVars2 == 'gasvolume')] = 0.0 # no gas 

# Set initial concentration values (from point S-1)
inistates = ['Ca+2.tot',  'Mg+2.tot',
             'Na+.tot', 'K+.tot', 'F-.tot', 'Cl-.tot', 'SO4-2.tot']

IN2[0][np.where(np.isin(InVars2,inistates))] = df_data.loc['S-1',inistates].values

# We obtain the target boundary values from Res_Simulation
bndvals = ['T','CO2[g].logact']

IN2[0][np.where(np.isin(InVars2,bndvals))] = Res_Simulation.loc['F-6',bndvals].values
IN2[0][np.where(np.isin(InVars2,'deltaSIcalcite'))] = Res_Simulation.loc['F-6','Calcite[s].si']

# Calculate equilibrium conditions for initial situation (Point A in chapter 4.1)
OUT = pO2.set_and_calculate(IN2)
Res_Sim2 = pd.DataFrame(OUT,columns=OutVars2)

# Need to add the deltaSIcalcite value to the Calcite[s].si value in the output (Orchestra does not do this automatically)
Res_Sim2['Calcite[s].si'] += Res_Simulation.loc['F-6','Calcite[s].si']

Res_Sim2.rename(index={0: 'S-1 to F-6'}, inplace=True)

print("Equilibrate sample from S-1 to F-6")
print(Res_Sim2[['T','pH','Ca+2.con', 'HCO3-.con', 'CO2[g].con', 'Calcite[s].si', 'Calcite[s].tot', 'CO2[g].logact']])

print("Results from first simulation")
print(Res_Simulation[['T','pH','Ca+2.con', 'HCO3-.con', 'CO2[g].con', 'Calcite[s].si', 'Calcite[s].tot', 'CO2[g].logact']])

Equilibrate sample from S-1 to F-6
                     T        pH  Ca+2.con  HCO3-.con  CO2[g].con  \
S-1 to F-6  287.549988  8.233166   0.00275   0.003107    0.000909   

            Calcite[s].si  Calcite[s].tot  CO2[g].logact  
S-1 to F-6       0.903334        0.000709      -3.041655  
Results from first simulation
                 T    pH  Ca+2.con  HCO3-.con  CO2[g].con  Calcite[s].si  \
Sample                                                                     
S-1     297.450012  7.32  0.003413   0.004458    0.012252       0.369630   
F-6     287.549988  8.25  0.002523   0.003229    0.000909       0.903334   

        Calcite[s].tot  CO2[g].logact  
Sample                                 
S-1                0.0      -1.911809  
F-6                0.0      -3.041655  


```{exercise} Explain the above results
1. Carefully evaluate the python code, explain what each of the code steps do.
2. Analyse the outcomes of the simulation and explain why simulated concentrations (S-1 to F-6) are different from F-6.
```

## Assignment
In the above we have analysed two of the samples shown in Lorah and Herman (1988). Your task is to analyse all other results in a similar way. For this assignment you need to do the following:

1. Import all data from Table 1 in Lorah and Herman.
2. Translate all values to K and mg/l where relevant.
3. Add additional information from Lorah and Herman so you can make nice plots. For example, add the distance along the stream which you can estimate from the map shown in Figure 1.
4. Create plots of the raw data similar to figures 2 and 3 to better understand the data.
5. Evaluate all measurements using Orchestra with the *chemistry1.inp* file.
6. Carefully evaluate the results, make plots for the CO2[g].con, CO2[g].logact, Calcite[s].si etc. so that you understand how the chemistry changes along the stream.
7. Set up a series of Orchestra calculations using the *chemistry2.inp* file in order to estimate the changes in the amounts of HCO3-, Calcite and Ca+2 along the stream.
8. Finally, use the Orchestra to estimate the total amount of Calcite that can precipitate from the samples when they would be brought to equilibrium with the atmosphere in the laboratory at 25 $^\text{o} \text{C}$.

$\text{CO}_2\text{[g]}$

